# Step 1: Extract Data
### Import libraries, set paths & constants

In [1]:
import csv
import numpy as np
import os
import pandas as pd
import sys
import time

from IPython.display import display
from collections import defaultdict
from pandas import DataFrame

In [2]:
# Set these as appropriate
MIMIC_PATH = '/Users/Mark/Downloads/MIMIC Data/Original'
VARIABLE_MAP_FILE = '/Users/Mark/Documents/GitHub/MIMIC-Machine-Learning/Reference Files/mimic3_map.csv'
OUTPUT_PATH = '/Users/Mark/Downloads/MIMIC Data/Processed'

NB_ROWS_CHARTEVENTS = 263201376
NB_ROWS_LABEVENTS = 27872576
NB_ROWS_OUTPUTEVENTS = 4349340

try:
    os.makedirs(OUTPUT_PATH)
except:
    pass

### Filter data by various criteria

In [3]:
pats = pd.read_csv(os.path.join(MIMIC_PATH, 'PATIENTS_DATA_TABLE.csv'))
pats = pats[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD']]
pats.DOB = pd.to_datetime(pats.DOB)
pats.DOD = pd.to_datetime(pats.DOD)

admits = pd.read_csv(os.path.join(MIMIC_PATH, 'ADMISSIONS_DATA_TABLE.csv'))
admits = admits[['SUBJECT_ID', 'HADM_ID', 'DEATHTIME', 'ETHNICITY', 'DIAGNOSIS']]
admits.DEATHTIME = pd.to_datetime(admits.DEATHTIME)

stays  = pd.read_csv(os.path.join(MIMIC_PATH, 'ICUSTAYS_DATA_TABLE.csv'))
print('START:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0])
stays = stays.ix[(stays.FIRST_WARDID == stays.LAST_WARDID) & (stays.FIRST_CAREUNIT == stays.LAST_CAREUNIT)]
stays = stays[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE', 'INTIME', 'OUTTIME', 'LOS']]

stays.INTIME = pd.to_datetime(stays.INTIME)
stays.OUTTIME = pd.to_datetime(stays.OUTTIME)
print('REMOVE ICU TRANSFERS:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0])

stays = stays.merge(admits, how='inner', left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])
stays = stays.merge(pats, how='inner', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])

START: 61532 57786 46476
REMOVE ICU TRANSFERS: 55830 52834 43277


In [4]:
stays['AGE'] = (stays.INTIME - stays.DOB).apply(lambda s: s / np.timedelta64(1, 's')) / 60./60/24/365
stays.AGE[stays.AGE<0] = 90
stays = stays.ix[stays.AGE >= 18]
print('REMOVE PATIENTS AGE < 18:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0])

REMOVE PATIENTS AGE < 18: 47702 44815 35380


C:\Users\Mark\Miniconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
stays = stays.ix[stays.LOS >= .25]
print('REMOVE PATIENTS LOS < 6hrs:', stays.ICUSTAY_ID.unique().shape[0], stays.HADM_ID.unique().shape[0], stays.SUBJECT_ID.unique().shape[0])

REMOVE PATIENTS LOS < 6hrs: 47278 44540 35165


### Structure data by episodes: 1 folder per ICU Stay

In [6]:
# Uncomment when testing - randomly sample 1000 stays rows
stay_idx = np.random.randint(0, high=stays.shape[0], size=1000)
stays = stays.iloc[stay_idx]

#### Create folders for each episode, each with stays.csv and observations.csv

In [7]:
OBS_HEADER = [ 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VARIABLE', 'VALUE', 'VALUEUOM', 'MIMIC_LABEL' ]
nb_subjects = stays.SUBJECT_ID.unique().shape[0]
t0 = time.time()
for i, subject_id in enumerate(stays.SUBJECT_ID.unique()):
    sys.stdout.write('\rSUBJECT {0} of {1}...'.format(i, nb_subjects))
    dn = os.path.join(OUTPUT_PATH, str(subject_id))
    try:
        os.makedirs(dn)
    except:
        pass
    
    stays.ix[stays.SUBJECT_ID == subject_id].sort_values(by='INTIME').to_csv(os.path.join(dn, 'stays.csv'), index=False)
    f = open(os.path.join(dn, 'observations.csv'), 'w')
    f.write(','.join(OBS_HEADER) + '\n')
    f.close()
        
tf = time.time()
print ('DONE in %.1f seconds' % (tf-t0))

SUBJECT 971 of 972...DONE in 8.9 seconds


#### Build mapping dataframe
This imports the map file with the following criteria:  
1) Drop labels not mapped to a variable  
2) Unsure variables (denoted by the 'ZZZZ' prefix)  
2) Variables with 0 counts in respective events tables

In [8]:
mp = DataFrame.from_csv(VARIABLE_MAP_FILE, index_col=None).fillna('').astype(str)
mp.COUNT = pd.to_numeric(mp.COUNT)
mp = mp.ix[(mp.VARIABLE != '') & (mp.COUNT>0)]
mp = mp.ix[mp.VARIABLE.apply(lambda s: not s.startswith('ZZZZ'))]
mp = mp.set_index('ITEMID')

#### Build observations files

In [9]:
subject_ids = set([ str(s) for s in stays.SUBJECT_ID.unique() ])

tables = [ 'chartevents'] #, 'labevents' ] #, 'outputevents' ]
nb_rows = [ NB_ROWS_CHARTEVENTS] #, NB_ROWS_LABEVENTS ] #, NB_ROWS_OUTPUTEVENTS ]
t0 = time.time()

for table, nbr in zip(tables, nb_rows):
    r = csv.DictReader(open(os.path.join(MIMIC_PATH, table.upper() + '_DATA_TABLE.csv'), 'r'))
    curr_subject_id = ''
    last_write_no = 0
    last_write_nb_rows = 0
    last_write_subject_id = ''
    curr_obs = []
    for i, row_in in enumerate(r):
        if i > 1000000:
            break
        if last_write_no != '':
            sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...last write '
                             '({3}) {4} rows for subject {5}'.format(table, i, nbr, last_write_no,
                                                                     last_write_nb_rows, last_write_subject_id))
        else:
            sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...'.format(table, i, nbr))
        
        subject_id = row_in['SUBJECT_ID']
        itemid = row_in['ITEMID']
        
        if subject_id not in subject_ids or itemid not in mp.index:
            continue
            
        row_out = { 'SUBJECT_ID': subject_id,
                    'HADM_ID': row_in['HADM_ID'],
                    'CHARTTIME': row_in['CHARTTIME'],
                    'VARIABLE': mp.VARIABLE[row_in['ITEMID']],
                    'MIMIC_LABEL': mp.LABEL[row_in['ITEMID']],
                    'VALUE': row_in['VALUE'],
                    'VALUEUOM': row_in['VALUEUOM'] }
        
        try:
            row_out['ICUSTAY_ID'] = row_in['ICUSTAY_ID']
        except:
            row_out['ICUSTAY_ID'] = ''

        if curr_subject_id != '' and curr_subject_id != subject_id:
            last_write_no += 1
            last_write_nb_rows = len(curr_obs)
            last_write_subject_id = curr_subject_id
            fn = os.path.join(OUTPUT_PATH, str(curr_subject_id), 'observations.csv')
            w = csv.DictWriter(open(fn, 'a', newline=''), fieldnames=OBS_HEADER, quoting=csv.QUOTE_MINIMAL)
            w.writerows(curr_obs)
            curr_obs = []

        curr_obs.append(row_out)
        curr_subject_id = subject_id

    if curr_subject_id != '':
        last_write_no += 1
        last_write_nb_rows = len(curr_obs)
        last_write_subject_id = curr_subject_id
        fn = os.path.join(OUTPUT_PATH, str(curr_subject_id), 'observations.csv')
        w = csv.DictWriter(open(fn, 'a', newline=''), fieldnames=OBS_HEADER, quoting=csv.QUOTE_MINIMAL)
        w.writerows(curr_obs)
        curr_obs = []
    
    del r
    sys.stdout.write('\rprocessing {0}: ROW {1} of {2}...last write '
                     '({3}) {4} rows for subject {5}'.format(table, i, nbr, last_write_no,
                                                             last_write_nb_rows, last_write_stay_id))
    tf = time.time()
    print('DONE after %d seconds' % tf-t0)

processing chartevents: ROW 1000000 of 263201376...last write (251) 37 rows for subject 9402

NameError: name 'last_write_stay_id' is not defined